In [2]:

with open('../Harry_Potter_all_books_preprocessed.txt','r',encoding='utf-8') as f:
    text = f.read()
len(text)

5991293

In [3]:
print(text[:1000])

THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much .They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense .Mr Dursley was the director of a firm called Grunnings which made drills .He was a big beefy man with hardly any neck although he did have a very large mustache .Mrs Dursley was thin and blonde and had nearly twice the usual amount of neck which came in very useful as she spent so much of her time craning over garden fences spying on the neighbors .The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere .The Dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it .They didnt think they could bear it if anyone found out about the Potters .Mrs Potter was Mrs Dursleys sister but they hadnt met for several years in f

In [4]:
characters = sorted(list(set(text)))
"".join(characters)

' !.0123456789?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz~‘•■□'

In [13]:
vocab_size = len(characters)

In [6]:
#encoder : takes a string/text, and gives out indexes of the characters from our vocab
#decoder: reverse of encoding, takes in a list of numbers/indexes, and gives back original string

## tradeoff between vocabulary size and sequence/token size
import tiktoken
encoder = tiktoken.get_encoding('gpt2')
encoder.n_vocab

50257

In [7]:
enc = {s:i for i,s in enumerate(characters) }
dec = {i:s for i,s in enumerate(characters) }
encoder = lambda meow : [enc[i] for i in meow]
decoder = lambda meow2 : "".join([dec[i] for i in meow2])

In [8]:
encoder('meow')

[52, 44, 54, 62]

In [9]:
import torch
data = torch.tensor(encoder(text),dtype=torch.long)
print(data[:1000])

tensor([33, 21, 18,  0, 15, 28, 38,  0, 36, 21, 28,  0, 25, 22, 35, 18, 17,  0,
        26, 57,  0, 40, 53, 43,  0, 26, 57, 58,  0, 17, 60, 57, 58, 51, 44, 64,
         0, 54, 45,  0, 53, 60, 52, 41, 44, 57,  0, 45, 54, 60, 57,  0, 29, 57,
        48, 61, 44, 59,  0, 17, 57, 48, 61, 44,  0, 62, 44, 57, 44,  0, 55, 57,
        54, 60, 43,  0, 59, 54,  0, 58, 40, 64,  0, 59, 47, 40, 59,  0, 59, 47,
        44, 64,  0, 62, 44, 57, 44,  0, 55, 44, 57, 45, 44, 42, 59, 51, 64,  0,
        53, 54, 57, 52, 40, 51,  0, 59, 47, 40, 53, 50,  0, 64, 54, 60,  0, 61,
        44, 57, 64,  0, 52, 60, 42, 47,  0,  2, 33, 47, 44, 64,  0, 62, 44, 57,
        44,  0, 59, 47, 44,  0, 51, 40, 58, 59,  0, 55, 44, 54, 55, 51, 44,  0,
        64, 54, 60, 43,  0, 44, 63, 55, 44, 42, 59,  0, 59, 54,  0, 41, 44,  0,
        48, 53, 61, 54, 51, 61, 44, 43,  0, 48, 53,  0, 40, 53, 64, 59, 47, 48,
        53, 46,  0, 58, 59, 57, 40, 53, 46, 44,  0, 54, 57,  0, 52, 64, 58, 59,
        44, 57, 48, 54, 60, 58,  0, 41, 

In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [28]:
torch.manual_seed(1337)
batch_size = 24 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([24, 8])
tensor([[64, 54, 60,  0, 50, 53, 54, 62],
        [55, 40, 57, 50,  0,  2, 28, 53],
        [59, 44, 57, 41, 44, 44, 57,  0],
        [44, 40, 42, 47, 44, 57, 58,  0],
        [48, 58,  0, 40, 58,  0, 48, 59],
        [40, 43,  0, 53, 44, 61, 44, 57],
        [31, 54, 53,  0, 40, 53, 43,  0],
        [62, 48, 59, 47,  0, 59, 47, 44],
        [ 0, 59, 47, 40, 53,  0, 40, 53],
        [64,  0, 40, 53, 64, 52, 54, 57],
        [ 0, 62, 48, 65, 40, 57, 43, 58],
        [60, 51, 43, 53, 59,  0, 47, 40],
        [58, 40, 48, 43,  0, 21, 40, 57],
        [43,  0, 21, 40, 57, 57, 64,  0],
        [54, 60, 58, 44,  0, 13, 21, 40],
        [48, 53,  0, 59, 47, 44,  0, 58],
        [48, 54, 53, 58,  0, 54, 45,  0],
        [51, 40, 48, 53, 51, 64,  0, 59],
        [59,  0, 62, 47, 54,  0, 62, 40],
        [55, 44, 53, 52, 54, 60, 59, 47],
        [46, 60, 58,  0, 47, 40, 58,  0],
        [ 0, 58, 44, 44, 52, 44, 43,  0],
        [43, 48, 43, 53, 59,  0, 59, 44],
      

In [29]:
## an embedding table is a simple lookup table for word embeddings using indices.
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # of size vocab_size * vocab_size
        # every value from our input is going to retrieve a row from the embedding table

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) batch * time * channel / 4 * 8 * 65
        # logits are the scores/predictions for the next character

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # 2-D
            targets = targets.view(B*T) # 1-D
            loss = F.cross_entropy(logits, targets) # neg log likelihood
            ## pytorch wants B*C*T
            ## expected loss - > ln(1/vocab_size) -> ln(1/65) = 4.17

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #pluck out last element in time dimension as they are our predictionss
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) #one element prediction for each batch 
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long) # 1 * 1 tensor of zeroth index i.e apostrophe
print(decoder(m.generate(idx, max_new_tokens=100)[0].tolist()))
# untrained model gives garbage :(


torch.Size([192, 71])
tensor(4.9205, grad_fn=<NllLossBackward0>)
 3yhb■rnCYI1Z‘ZjKrtYdC2dB5iWFK!vIH4VBji6Nes8szMeFzq5ic5■C□55VED?VB9sYz‘OAtayh6Mes8‘Zj4nQj?r2TEi5rIFqB


In [30]:
#better optiimzers?

# Experiment with SGD + Momentum: If Adam doesn't perform as well as expected, try SGD with momentum. It can sometimes provide better generalization.

# RMSprop: If you have non-stationary data or find that the gradients vary significantly, RMSprop can be beneficial.

In [31]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [37]:
batch_size = 32
for steps in range(5000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() # update params

print(loss.item())

2.4069924354553223


In [38]:
print(decoder(m.generate(idx, max_new_tokens=100)[0].tolist()))

 warinethi□.Tan am le Whe g t arentt we Yo■n hily thE2.GGre It e y ffoll5ite s lant dl wach ouain wli


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
torch.manual_seed(4)


with open('../Harry_Potter_all_books_preprocessed.txt','r',encoding='utf-8') as f:
    text = f.read()
    
#encoder : takes a string/text, and gives out indexes of the characters from our vocab
#decoder: reverse of encoding, takes in a list of numbers/indexes, and gives back original string

enc = {s:i for i,s in enumerate(characters) }
dec = {i:s for i,s in enumerate(characters) }
encoder = lambda meow : [enc[i] for i in meow]
decoder = lambda meow2 : "".join([dec[i] for i in meow2])

data = torch.tensor(encoder(text),dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

@torch.no_grad() ##  Disables gradient computation, which reduces memory usage and speeds up computations during evaluation.

In [ ]:
# ensures that layers like dropout and batch normalization are in evaluation mode, providing more accurate loss measurements for both training and validation sets.
# In contrast, the training loop is focused on parameter updates and does not ensure such an accurate evaluation context.
# computations are efficient and do not interfere with the training process via torch.no_grad()
# estimate_loss function computes the average loss over multiple mini-batches, 

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('../Harry_Potter_all_books_preprocessed.txt','r',encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.8242, val loss 4.8388
step 300: train loss 2.7296, val loss 2.7216
step 600: train loss 2.4657, val loss 2.4579
step 900: train loss 2.4228, val loss 2.4130
step 1200: train loss 2.4004, val loss 2.3908
step 1500: train loss 2.4045, val loss 2.3753
step 1800: train loss 2.3940, val loss 2.3806
step 2100: train loss 2.3915, val loss 2.3691
step 2400: train loss 2.3863, val loss 2.3693
step 2700: train loss 2.3903, val loss 2.3677
 g tos hand aryou bes theeato hoy beeryey osnontauly bely y s boust Prs .Intand Nad het Antid o iker u fom mad .Ivear w .Butan id honinor tt harcKOhera us nthadn .If wh herys .Spkng simma bey Harondmbers ncix haord iuns we t orinasclur lfis beld he w nokfis ?Thisupppimay benowhiowagldeinghig gonthe he adSouerint wrick ve hingehad he hin Mreaimme lind penthus hin ioatid y ch s n thond !My t they u lineadorhtevee .He yst homu t tithoon ur fed indgrowous ut Wek walate wimbures frmip chewead wes and 


In [ ]:
## math trick for self attention
## for a position,calculate average of all preceding vectors and at that position via channels
## information should only flow backwards from the latest token
## take in channels info and go backwards

In [69]:
# consider the following toy example:

torch.manual_seed(4)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x

tensor([[[-0.9414,  1.2632],
         [-0.1838,  0.1505],
         [ 0.1075, -0.2780],
         [-2.6021,  0.6245],
         [-0.8684, -0.2051],
         [ 0.3976,  0.6699],
         [-0.0537,  0.0467],
         [-1.7671, -2.1205]],

        [[ 1.5191, -0.6682],
         [ 0.0031, -0.1535],
         [ 1.1396, -0.2302],
         [ 1.1877,  0.7677],
         [-0.7588, -0.1853],
         [-0.8558, -0.2346],
         [-0.4215,  0.8488],
         [-0.6776, -0.9445]],

        [[-0.4815,  1.2434],
         [ 2.3693,  0.2829],
         [-0.2345,  1.6892],
         [ 0.2716, -0.1365],
         [-0.6948, -1.3186],
         [-0.9694,  0.6403],
         [ 0.8201, -0.9151],
         [-2.1437,  1.4072]],

        [[-0.0263,  2.7204],
         [-0.5955,  0.9871],
         [ 1.0861,  0.0610],
         [ 0.0417,  0.6783],
         [-0.8952, -1.0143],
         [-0.2429, -1.5727],
         [ 1.3940, -0.1941],
         [ 0.0048, -1.3165]]])

In [73]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        print(xprev)
        xbow[b,t] = torch.mean(xprev, 0)
print(xbow)

tensor([[-0.9414,  1.2632]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780],
        [-2.6021,  0.6245]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780],
        [-2.6021,  0.6245],
        [-0.8684, -0.2051]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780],
        [-2.6021,  0.6245],
        [-0.8684, -0.2051],
        [ 0.3976,  0.6699]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780],
        [-2.6021,  0.6245],
        [-0.8684, -0.2051],
        [ 0.3976,  0.6699],
        [-0.0537,  0.0467]])
tensor([[-0.9414,  1.2632],
        [-0.1838,  0.1505],
        [ 0.1075, -0.2780],
        [-2.6021,  0.6245],
        [-0.8684, -0.2051],
        [ 0.3976,  0.6699],
        [-0.0537,  0.0467],
        [-1.7

In [71]:
xbow[0]

tensor([[-0.9414,  1.2632],
        [-0.5626,  0.7069],
        [-0.3392,  0.3786],
        [-0.9050,  0.4400],
        [-0.8976,  0.3110],
        [-0.6818,  0.3708],
        [-0.5920,  0.3245],
        [-0.7389,  0.0189]])

In [74]:
(-0.9414-0.1838+0.1075)/3

-0.33923333333333333

In [78]:
## lets use mathematical trick of self attention
## mulitplying a normalized version of a lower left triangle of ones with our data can give
## us the average must easily
## batched matrix multiply 

In [77]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2[0]

tensor([[-0.9414,  1.2632],
        [-0.5626,  0.7069],
        [-0.3392,  0.3786],
        [-0.9050,  0.4400],
        [-0.8976,  0.3110],
        [-0.6818,  0.3708],
        [-0.5920,  0.3245],
        [-0.7389,  0.0189]])

In [79]:
## softmax version
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) ## tokens from past cant communicate as we set them to -inf
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)
## weighted aggregation of the past tokens 

True

In [ ]:
## self-attention: get information about tokens that are interesting to oneself
## every token will give out a query and a key
## the way we form relations / affinities between tokens is by a dot product between queries and keys
## that dot product is wei
## query -> what i am looking for
## key -> what do i contain

In [80]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) ## removin this line allows all tokens to comm with each other
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [84]:
wei[0][7] 
## becomes data dependent
## 8th token knows what it contents and its position

tensor([0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391],
       grad_fn=<SelectBackward0>)

In [85]:
## self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", 
#the queries still get produced from x,
#but the keys and values come from some other, external source (e.g. an encoder module)
## "Scaled" attention additional divides wei by 1/sqrt(head_size). 
#This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [87]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [88]:
wei

tensor([[[ 1.9857, -1.7370,  0.4189,  0.2985, -0.5451,  0.4942, -0.7267,
          -0.4810],
         [-0.6151,  0.7704,  0.1215,  0.1193, -1.0559, -0.1234,  0.3918,
          -0.2687],
         [ 0.4511,  0.6600,  0.8736, -0.5065,  0.8595,  0.2483,  0.7095,
          -0.3241],
         [ 1.4350, -0.5599,  1.2163, -0.0813,  1.7313,  0.3421, -0.3146,
          -0.9178],
         [-2.0204,  1.8716, -1.1214, -0.1317, -0.4320,  0.8461,  1.0991,
           1.8651],
         [ 1.0000,  0.5394,  0.9807, -0.0900,  0.7364,  1.3018,  1.4779,
           1.2385],
         [ 1.0542, -0.5249,  0.1258, -0.0781,  0.8236, -1.0546,  0.3601,
          -0.5679],
         [ 0.2587,  0.1620,  0.6471,  0.2837,  1.2641,  0.3890, -0.6218,
          -0.4601]],

        [[ 0.2958,  0.3852, -0.7456,  0.0486, -0.1722, -0.3054,  0.8299,
           0.4364],
         [-0.8550,  0.2635,  1.0761,  0.9544,  0.7529, -0.9505,  0.2712,
          -0.7474],
         [-0.9295, -0.1556, -0.0649, -0.3967, -0.1137, -1.0016,  0.5

In [86]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
## only self atteion with feedforward, no cross attention
## a decoder only transformer (triangular mask for attention)
## pre tuning of chatgpt: training on a chunk of internet text for the model to spit out the same